# Practica once

Grupo 14:
* Joaquín Ibáñez Penalva
* Aurora Zuoris

Para la realización de esta práctica se usará la librería de numpy, pandas, matplotlib, y sklearn.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('wisconsin diagnostic breast cancer.csv')
X = data.iloc[:, 2:].values
y = data.iloc[:, 1].values

X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

## Ejercicio uno

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

# Crear el modelo de árbol de decisión
model = DecisionTreeClassifier(random_state=42)

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Predecir la clase de los datos de test
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
acc = accuracy_score(y_test, y_pred)
acc2 = cross_val_score(model, X, y, cv=10).mean()
print(f"Accuracy del modelo por defecto: {acc:.3f}")
print(f"Accuracy del modelo con validación cruzada: {acc2:.3f}")

La validación cruzada lo que hace es hacer la división de datos en train y test varias veces, y calcular la media de los resultados. Esto es útil para evitar que el resultado dependa de la partición de los datos.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import recall_score

# Definir los valores de los parámetros a explorar
param_grid = {
    'max_depth': range(1, 10),
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'random'],
}

# Crear el objeto GridSearchCV
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid=param_grid, scoring='recall', cv=10)
y_train_bool = y_train == 'M'
# Entrenar el objeto GridSearchCV
grid_search.fit(X_train, y_train_bool)


# Obtener los mejores parámetros y la accuracy correspondiente
best_params = grid_search.best_params_
best_acc = grid_search.best_score_

print(f"Mejores parámetros: {best_params}")
print(f"Accuracy con los mejores parámetros: {best_acc:.3f}")

Como se puede comprobar, fijandose en poner los mejores parámetros el accuracy es de 0.96% practicamente, mientras que con un árbol por defecto es de 0.93% usando validación cruzada.

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
tree = DecisionTreeClassifier(**best_params, random_state=42)
tree.fit(X_train, y_train)

y_pred = tree.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=tree.classes_)
disp.plot()

Al ser un tema medico, nos hemos fijado en el mejor arbol teniendo en cuenta que tenga la menor tasa de falsos negativos posible, ya que es peor que un paciente tenga cancer y no se le diagnostique, que un paciente que no tenga cancer y se le diagnostique. Esto queda demostrado con la matriz de confusión, que da 0 en falsos negativos.

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

# Mostrar gráficamente los árboles obtenidos con los parámetros por defecto y los mejores hiperparámetros
plt.figure(figsize=(15,15))
plt.subplot(2,1,1)
plot_tree(tree, filled=True)
plt.title("Árbol de decisión con mejores hiperparámetros")
plt.subplot(2,1,2)
plot_tree(model, filled=True)
plt.title("Árbol de decisión con parámetros por defecto")
plt.show()

Además de ser mejor el arbol con los mejores parámetros, es más sencillo de interpretar, ya que tiene menos nodos y menor profundidad.

## Ejercicio dos

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [6]:
dt = DecisionTreeClassifier(random_state=42).fit(X_train, y_train)
print(f'Decision tree score: {dt.score(X_test, y_test):.2%}')

Decision tree score: 94.74%


In [9]:
used_features = dt.feature_importances_ > 0
used_count = np.sum(used_features)
used_count, used_features

(12,
 array([False,  True, False, False, False, False, False,  True, False,
        False, False, False, False,  True,  True, False,  True,  True,
        False,  True,  True,  True,  True, False,  True, False, False,
         True, False, False]))

In [11]:
X_train_tree = X_train[:, used_features]
X_test_tree = X_test[:, used_features]

svc = SVC(random_state=42).fit(X_train_tree, y_train)
print(f'SVM score: {svc.score(X_test_tree, y_test):.2%}')

SVM score: 95.61%


In [13]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

selector = SelectKBest(mutual_info_classif, k=used_count)
selector.fit(X_train, y_train)
X_train_mut = selector.transform(X_train)
X_test_mut = selector.transform(X_test)

In [14]:
svc2 = SVC(random_state=42).fit(X_train_mut, y_train)
print(f'SVM score: {svc2.score(X_test_mut, y_test):.2%}')

SVM score: 94.74%


Se ve que el uso del arbol de decision para seleccionar las características da un resultado marginalmente (95.61% vs 94.74%, menos de 1% de differencia) mejor que el uso de filtrado mediante información mutua.

Para llegar a una conclusión más informativa, se utilizará cross validation para obtener el rendimiento medio de cada método.

In [17]:
from sklearn.model_selection import cross_val_score

X_tree = X[:, used_features]
X_mut = selector.transform(X)

scores1 = cross_val_score(svc, X_tree, y, cv=10)
scores2 = cross_val_score(svc2, X_mut, y, cv=10)

print(f'Desicion trees: {scores1.mean():.2%} ± {scores1.std():.2%}')
print(f'SVM: {scores2.mean():.2%} ± {scores2.std():.2%}')

Desicion trees: 93.85% ± 1.79%
SVM: 91.39% ± 2.88%


Por último se utiliza un test de T de Student para ver si la diferencia de rendimiento es significativa.

In [32]:
from numpy.random import seed
seed(42)
# t-test mediante solo numpy: https://stackoverflow.com/a/44763186
diff = scores2 - scores1

t = diff.mean() / diff.std(ddof=1) * np.sqrt(len(diff))
s = np.random.standard_t(len(diff), size=10_000_000)
p = np.sum(s<t) / float(len(s))
prob = 2 * min(p, 1-p)
print(f'p-value: {prob:.2%}')

p-value: 0.84%


El p-valor es de 0.84%, por lo que se puede concluir que la diferencia de rendimiento entre estos dos métodos de selección de característica es significativa.
De forma que usar el arbol de decisión para seleccionar las características es mejor que usar filtrado mediante información mutua.
Esto se puede razonar con que el uso de un arbol de decisión es más computacionalmente costoso, por lo que se puede esperar que de mejores resultados.
Además, la filtración no tiene en cuenta la relación entre las características, mientras que el arbol de decisión sí.

Otra perspectiva que podemos utilizar para este análisis es ver en qué concuerdan y en qué difieren los resultados de ambos métodos:

In [50]:
tree_features = set(np.where(used_features)[0])
filter_features = set(np.where(selector.get_support())[0])

common = tree_features & filter_features
common_names = ", ".join(data.columns[2:][list(common)])

tree_names = ", ".join(data.columns[2:][list(tree_features - common)])
filter_names = ", ".join(data.columns[2:][list(filter_features - common)])

print(f"Los dos métodos concuerdan en {len(common)} características ({common_names})\n")
print(f"El árbol de decisión eligió además estas características: {tree_names}\n")
print(f"Y el método de filtrado eligió estas otras: {filter_names}\n")


Los dos métodos concuerdan en 5 características (Mean concave points, Standard Error area, Worst radius, Worst perimeter, Worst concave points)

El árbol de decisión eligió además estas características: Mean texture, Standard Error smoothness, Standard Error concavity, Standard Error concave points, Standard Error fractal dimension, Worst texture, Worst smoothness

Y el método de filtrado eligió estas otras: Mean radius, Mean perimeter, Mean area, Mean concavity, Standard Error radius, Worst area, Worst concavity

